Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [2]:
!pip install icecream
from random import random, seed
from itertools import product
import numpy as np
from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [3]:
UNIVERSE_SIZE = 100_000
NUM_SETS = 10_000
DENSITY = 0.3



data = [
    [100, 10, 0.2],
    [1000, 100, 0.2],
    [10000, 1000, 0.3],
    [100000, 10000, 0.1],
    [100000, 10000, 0.2],
    [100000, 10000, 0.3]
]

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

## Helper Functions

In [4]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [5]:

# Apply the greedy algorithm to the first set of LIST_OF_SETS
class TabuSearch:
    def __init__(self, list_of_sets, max_iterations, tabu_tenure, max_no_improve):
        self.list_of_sets = list_of_sets
        self.max_iterations = max_iterations
        self.tabu_tenure = tabu_tenure
        self.max_no_improve = max_no_improve
        self.tabu_list = []
        self.best_solution = None
        self.best_cost = float('inf')
        self.evaluations=0

    def _get_coverage(self, selected_sets):
        # Calculate the coverage of the selected sets
        sets, _ = self.list_of_sets
        covered = np.any(sets[selected_sets], axis=0)
        return covered

    def _fitness_function(self, selected_sets):
        # Evaluate the solution based on the total cost of the selected sets
        self.evaluations+=1
        covered_elements = self._get_coverage(selected_sets)
        num_uncovered = np.sum(~covered_elements)
        return cost(selected_sets), num_uncovered

    def _generate_initial_solution(self):
        # Greedy initialization: select sets that cover the most elements
        sets, _ = self.list_of_sets
        num_elements = sets.shape[1]
        uncovered_elements = np.ones(num_elements, dtype=bool)
        selected_sets = []

        while np.any(uncovered_elements):
            # Select the set that covers the most uncovered elements
            cover_count = np.sum(sets[:, uncovered_elements], axis=1)
            best_set = np.argmax(cover_count)
            selected_sets.append(best_set)
            uncovered_elements = uncovered_elements & ~sets[best_set]

        return selected_sets

    def _get_neighborhood(self, current_solution):
        # Generate neighborhood by adding or removing one set from the current solution
        sets, _ = self.list_of_sets
        num_sets = sets.shape[0]
        neighborhood = []

        # Try adding a new set
        for s in range(num_sets):
            if s not in current_solution:
                new_solution = current_solution + [s]
                neighborhood.append(new_solution)

        # Try removing a set
        for s in current_solution:
            new_solution = [i for i in current_solution if i != s]
            neighborhood.append(new_solution)

        return neighborhood

    def run(self):
        current_solution = self._generate_initial_solution()
        current_cost, num_uncovered = self._fitness_function(current_solution)
        best_solution = current_solution
        best_cost = current_cost
        no_improve_count = 0

        for iteration in range(self.max_iterations):
            # print(f"Iteration {iteration + 1}/{self.max_iterations}")
            neighborhood = self._get_neighborhood(current_solution)
            best_neigh_solution = None
            best_neigh_cost = float('inf')

            # Evaluate all neighbors
            for neighbor in neighborhood:
                if neighbor not in self.tabu_list:
                    neigh_cost, neigh_uncovered = self._fitness_function(neighbor)

                    # Only consider valid solutions that cover all elements
                    if neigh_uncovered == 0 and neigh_cost < best_neigh_cost:
                        best_neigh_solution = neighbor
                        best_neigh_cost = neigh_cost

            # If no valid neighbor found, stop
            if best_neigh_solution is None:
                break

            # Update current solution to best neighbor
            current_solution = best_neigh_solution
            current_cost = best_neigh_cost

            # Update tabu list
            self.tabu_list.append(current_solution)
            if len(self.tabu_list) > self.tabu_tenure:
                self.tabu_list.pop(0)

            # Update best solution if necessary
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost
                no_improve_count = 0
            else:
                no_improve_count += 1

            # Stop if no improvement for too long
            if no_improve_count >= self.max_no_improve:
                break

        self.best_solution = best_solution
        self.best_cost = best_cost
        return best_solution, best_cost, self.evaluations









In [7]:


max_iterations = 1000
tabu_tenure = 50
max_no_improve = 5

for mar in (data):
    SETS = np.random.random((mar[1], mar[0])) < mar[2]
    for s in range(mar[0]):
        if not np.any(SETS[:, s]):
            SETS[np.random.randint(mar[1]), s] = True
    COSTS = np.power(SETS.sum(axis=1), 1.1)

    tabu_search = TabuSearch((SETS,COSTS), max_iterations, tabu_tenure, max_no_improve)
    best_solution, best_cost, num_eval = tabu_search.run()
    ic(valid(best_solution), cost(best_solution), num_eval, len(best_solution))



ic| valid(best_solution): True
    cost(best_solution): 276.3830031082482
    num_eval: 30
    len(best_solution): 9
ic| valid(best_solution): True
    cost(best_solution): 5868.5884399713095
    num_eval: 497
    len(best_solution): 17
ic| valid(best_solution): True
    cost(best_solution): 107812.5502974177
    num_eval: 4997
    len(best_solution): 16
ic| valid(best_solution): True
    cost(best_solution): 1518872.4344240825
    num_eval: 49997
    len(best_solution): 60
ic| valid(best_solution): True
    cost(best_solution): 1734564.597877632
    num_eval: 89997
    len(best_solution): 32
ic| valid(best_solution): True
    cost(best_solution): 1774409.1297232155
    num_eval: 49997
    len(best_solution): 21
